# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> **NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY**

In [ ]:
#!pip install -qU ragas==0.2.10

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [15]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/family/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/family/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [17]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [18]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

#### 🎬 Study Note
Using 'cheap' model for generator, because it is good enough

### Unrolled SDG

In [19]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [20]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [21]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

#### 🎬 Study Note:
Summaries, Headlines, and Theme - are Ragas defaults; not something we specify/specified

In [22]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '1359ee'. Skipping!
Property 'summary' already exists in node '646249'. Skipping!
Property 'summary' already exists in node '332e1a'. Skipping!
Property 'summary' already exists in node '5faafc'. Skipping!
Property 'summary' already exists in node 'd31b7d'. Skipping!
Property 'summary' already exists in node '84b376'. Skipping!
Property 'summary' already exists in node 'cf9d25'. Skipping!
Property 'summary' already exists in node 'aa934c'. Skipping!
Property 'summary' already exists in node '5b3cc3'. Skipping!
Property 'summary' already exists in node '5e5abb'. Skipping!
Property 'summary' already exists in node 'd0e0fc'. Skipping!
Property 'summary' already exists in node '1c272b'. Skipping!
Property 'summary' already exists in node '4378cd'. Skipping!
Property 'summary' already exists in node '922c77'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/41 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '1359ee'. Skipping!
Property 'summary_embedding' already exists in node 'd31b7d'. Skipping!
Property 'summary_embedding' already exists in node '84b376'. Skipping!
Property 'summary_embedding' already exists in node '5faafc'. Skipping!
Property 'summary_embedding' already exists in node '332e1a'. Skipping!
Property 'summary_embedding' already exists in node '646249'. Skipping!
Property 'summary_embedding' already exists in node '922c77'. Skipping!
Property 'summary_embedding' already exists in node '5b3cc3'. Skipping!
Property 'summary_embedding' already exists in node 'cf9d25'. Skipping!
Property 'summary_embedding' already exists in node '1c272b'. Skipping!
Property 'summary_embedding' already exists in node 'd0e0fc'. Skipping!
Property 'summary_embedding' already exists in node '4378cd'. Skipping!
Property 'summary_embedding' already exists in node '5e5abb'. Skipping!
Property 'summary_embedding' already exists in node 'aa934c'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 39, relationships: 478)

#### 🎬 Study Note
Graph did not render, but I imagine it would just look like spaghetti anyway with 39 nodes and 478 relationships.

We can save and load our knowledge graphs as follows.

In [23]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 39, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [25]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [26]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

#### ✅ Answer #1
Based on breakout room discussion, plus [Ragas documentation](https://docs.ragas.io/en/stable/concepts/test_data_generation/rag/#multi-hop-query)

Specific vs Abstract:
Specific Query: user queries that require only fact-based retrieval
Abstract Query: user queries that require interpretation or synthesis

Single vs multi-hop:
Single-Hop: user query that only requires retrieving data from one (single) data source / document
Multi-Hop: user query that requires information from two or more sources, and probably reasoning steps to determine the appropriate sources

Finally, we can use our `TestSetGenerator` to generate our testset!

In [27]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How are academic years defined and differentia...,"[Chapter 1 Academic Years, Academic Calendars,...",An academic year must be defined for every eli...,single_hop_specifc_query_synthesizer
1,What is the significance of Chapter 3 in relat...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
2,What is the Fedral Work-Study program and how ...,[Non-Term Characteristics A program that measu...,The Federal Work-Study (FWS) program is a Titl...,single_hop_specifc_query_synthesizer
3,Pell Grant how it work?,[both the credit or clock hours and the weeks ...,The Pell Grant amount a student can get depend...,single_hop_specifc_query_synthesizer
4,What information does Volume 8 provide regardi...,[Disbursement Timing in Subscription-Based Pro...,Volume 8 explains that in subscription-based p...,single_hop_specifc_query_synthesizer
5,disbursement timing in subscription programs a...,[<1-hop>\n\nboth the credit or clock hours and...,"In the context of disbursement timing, the fir...",multi_hop_abstract_query_synthesizer
6,Include clinical work in standard term periods...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context explains that clinical work conduc...,multi_hop_abstract_query_synthesizer
7,whats the disbursement timing in subscription-...,[<1-hop>\n\nboth the credit or clock hours and...,"In subscription-based programs, the first two ...",multi_hop_abstract_query_synthesizer
8,Volume 8 mispelled?,[<1-hop>\n\nInclusion of Clinical Work in a St...,The context refers to Volume 8 in relation to ...,multi_hop_specific_query_synthesizer
9,so like in chapter 2 it talks about academic y...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...","Based on the information from chapter 2, acade...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [28]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node 'ec9627'. Skipping!
Property 'summary' already exists in node '9749ec'. Skipping!
Property 'summary' already exists in node '09dc5c'. Skipping!
Property 'summary' already exists in node '469d4c'. Skipping!
Property 'summary' already exists in node '0e3edc'. Skipping!
Property 'summary' already exists in node '9eb721'. Skipping!
Property 'summary' already exists in node 'cd98f4'. Skipping!
Property 'summary' already exists in node '7e4411'. Skipping!
Property 'summary' already exists in node 'cb982a'. Skipping!
Property 'summary' already exists in node '2e6723'. Skipping!
Property 'summary' already exists in node '178a7e'. Skipping!
Property 'summary' already exists in node 'e56dda'. Skipping!
Property 'summary' already exists in node '24b520'. Skipping!
Property 'summary' already exists in node '915964'. Skipping!


...

Property 'summary_embedding' already exists in node 'ec9627'. Skipping!
Property 'summary_embedding' already exists in node '9749ec'. Skipping!
Property 'summary_embedding' already exists in node 'cd98f4'. Skipping!
Property 'summary_embedding' already exists in node '09dc5c'. Skipping!
Property 'summary_embedding' already exists in node '469d4c'. Skipping!
Property 'summary_embedding' already exists in node '7e4411'. Skipping!
Property 'summary_embedding' already exists in node '0e3edc'. Skipping!
Property 'summary_embedding' already exists in node '9eb721'. Skipping!
Property 'summary_embedding' already exists in node 'cb982a'. Skipping!
Property 'summary_embedding' already exists in node 'e56dda'. Skipping!
Property 'summary_embedding' already exists in node '178a7e'. Skipping!
Property 'summary_embedding' already exists in node '2e6723'. Skipping!
Property 'summary_embedding' already exists in node '915964'. Skipping!
Property 'summary_embedding' already exists in node '24b520'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [29]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Whaat is the role of the School Participaton D...,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide specific details ...,single_hop_specifc_query_synthesizer
1,What is 34 CFR 668.3(a) about in terms of acad...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(a) relates to the minimum require...,single_hop_specifc_query_synthesizer
2,Could you explain the significance of Volume 8...,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,Is the Federal Work-Study (FWS) program subjec...,[Non-Term Characteristics A program that measu...,"No, the payment period is applicable to all Ti...",single_hop_specifc_query_synthesizer
4,How do accelerated progression effects influen...,[<1-hop>\n\nboth the credit or clock hours and...,Accelerated progression in clock-hour or non-t...,multi_hop_abstract_query_synthesizer
5,How do the defined payment periods relate to t...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The defined payment periods are part of the ac...,multi_hop_abstract_query_synthesizer
6,what are the academic calendars and regulatory...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",the academic calendars must define academic ye...,multi_hop_abstract_query_synthesizer
7,what program reqirements are in 34 CFR 668.3(b...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...","According to the context, 34 CFR 668.3(b) spec...",multi_hop_abstract_query_synthesizer
8,How do Volume 2 and Volume 8 relate to the inc...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Volume 2 provides guidance on the inclusion of...,multi_hop_specific_query_synthesizer
9,Considering the detailed requirements outlined...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The definitions and regulations in Volume 8 sp...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [44]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [ ]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

#### 🎬 Study Note:
It appears that this cell was inadvertantly run a second time between the first and second evaluations. Because the first run has 12 examples, but the 2nd run has 24. (See corresponding note in Activity #3 response.)

## Basic RAG Chain

Time for some RAG!


In [32]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


#### 🎬 Study Notes
Overlap is discussed more in Session 8

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [34]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [35]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [36]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [37]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

#### 🎬 Study Note
Above md cell appears to be incorrect; perhaps leftover from older cohort??


In [38]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [39]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [40]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available are:\n\n- Direct PLUS Loan or student Federal PLUS Loan  \n- Subsidized Federal Stafford Loans  \n- Unsubsidized Federal Stafford Loans  \n- Federal SLS Loans  \n- Federal PLUS Loans  \n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans  \n\nNote: Subsidized and Unsubsidized Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the Federal Family Education Loan (FFEL) Program before July 1, 2010; no new FFEL Program loans have been made since then.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [41]:
eval_llm = ChatOpenAI(model="gpt-4.1")

#### 🎬 Study Note

using more capable model for evals (so that the "grader knows more than the student")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [42]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `empathy_evaluator`:

#### ✅ Response to Activity #2
Please see detailed response in attached PDF here: [Activity2_response_evaluator_explanation.pdf](./Activity2_response_evaluator_explanation.pdf) 

## LangSmith Evaluation

#### 🎬 Study Note
using this md cell as a handy place to preserve the langsmith URL

Initial run:
View the evaluation results for experiment: 'earnest-meat-17' at:

https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/f76e0c37-88b6-4b5b-aa85-328dd0d8cc9e/compare?selectedSessions=25695b40-df30-4be2-9abc-089ccc6a1e13


In [43]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'earnest-meat-17' at:
https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/f76e0c37-88b6-4b5b-aa85-328dd0d8cc9e/compare?selectedSessions=25695b40-df30-4be2-9abc-089ccc6a1e13




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Appendix A and Appendix B relate to dis...,"Based on the provided context, Appendix B spec...",None,Appendix B provides detailed guidance on disbu...,0,0,0,5.782943,841b291b-2270-44be-8b2f-93c41db161b2,3e95651f-3133-44cc-9346-957913d6743a
1,Considering the inclusion of clinical work in ...,"Based on the provided context, the inclusion o...",None,The guidance in Volume 7 emphasizes that for c...,1,1,0,7.228194,8526a7f8-8f9a-4167-ae2d-48cea8bc1f32,26464662-c96f-42bf-937d-5908beb96716
2,Considering the detailed requirements outlined...,"Based on the provided context, Volume 8, parti...",None,The definitions and regulations in Volume 8 sp...,0,0,0,8.820087,4b273e46-8a37-4130-96c2-91b5842f1907,41effdfa-76ed-4e62-bd90-81b339f0bfc6
3,How do Volume 2 and Volume 8 relate to the inc...,I don't know.,None,Volume 2 provides guidance on the inclusion of...,0,0,0,0.664010,8c3dc6c0-4c6a-4b15-90b5-d156a4874044,1d2d0c77-75ad-4d12-a04e-f767fd6d8ff6
4,what program reqirements are in 34 CFR 668.3(b...,The program requirements in 34 CFR 668.3(b) fo...,None,"According to the context, 34 CFR 668.3(b) spec...",1,1,0,2.069959,3e70d909-8077-4c9c-9c76-32b369805b8a,7721f688-fbc4-4c67-9820-a76b1fcd96b0
5,what are the academic calendars and regulatory...,The academic calendars for Title IV programs m...,None,the academic calendars must define academic ye...,1,1,0,5.413760,4f4c8441-9b63-43c5-9974-bd06928ff6f1,730ee30a-70bc-4d29-9d60-e4978b6e7d78
6,How do the defined payment periods relate to t...,"Based on the provided context, the defined pay...",None,The defined payment periods are part of the ac...,1,1,0,5.845831,7f602c7b-a21c-4d7f-b869-c1c25a835f1e,3016f97f-8928-4638-bdd8-484f2d0c1933
7,How do accelerated progression effects influen...,Based on the provided context:\n\n- For **cloc...,None,Accelerated progression in clock-hour or non-t...,1,1,0,6.381556,a5268021-7dea-4b34-82f9-37682c87d1e5,c9773447-4383-444c-a2f0-5757407c5888
8,Is the Federal Work-Study (FWS) program subjec...,"No, the Federal Work-Study (FWS) program is no...",None,"No, the payment period is applicable to all Ti...",1,1,0,1.081625,5ac8e2f0-1375-46a1-9d69-b2788b184935,ac40c212-7fee-4902-9598-cbc4e8fc8fdf
9,Could you explain the significance of Volume 8...,Volume 8 provides important guidance on certai...,None,Inclusion of clinical work in a standard term ...,1,1,0,5.509593,e2b8b5ac-db66-42db-97e5-63fbcd936cec,70343b6d-6899-41ac-8c4c-1bd40081f297


## Dope-ifying Our Application - Empifying!

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [46]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [47]:
rag_documents = docs

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

#### ✅ Answer #2
There are tradeoffs.
Larger chunks can capture more complete ideas. This reduces the chance that important chunks will fail to be retrieved because they are too fragmented for the retriever to find the relevance. 

However, larger chunks can also increase noisiness, and LLMs can get distracted or lose focus if there is too much context. (Increased coverage; potentially decreased coherence.)

Larger chunks also cost (and potentially waste) more token usage.



In [49]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

#### ✅ Answer #3:
The "text-embedding-3-large" embedding model uses 3,072 dimensional vectors (twice as many as the 'small' one we've used before).
More dimensions means capturing more aspects of the data, creating better relationships between embeddings, and making retrieval more accurate.

A model with higher dimensionality is also better able to handle larger chunks, without overly compressing semantic meaning. Larger chunks allow for more robust context, as noted in Answer #2 above.

In short, better embedding should yield better retrieval quality. 

#### 🎬 Study Note:

You can also improve embedding models by  custom-training or fine-tuning them. 
This is more feasible than fine tuning an LLM (more bang for your buck).

In breakout, Can shared that Google has good embedding models, but more expensive than OpenAI.
Cohere makes good inexpensive models, and also good at re-ranking models (see Session 8). 

In [50]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [51]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [52]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [53]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for asking such an important question about loan options. Based on the information provided, here are the kinds of loans available:\n\n1. **Direct Subsidized Loans** – These loans are based on your financial need, calculated as the Cost of Attendance (COA) minus other financial aid. They have an annual maximum limit depending on your year and dependency status.\n\n2. **Direct Unsubsidized Loans** – These loans are available to students regardless of financial need and can be combined with subsidized loans to cover remaining costs.\n\n3. **Direct PLUS Loans** – These are loans that parents of dependent students can take out to cover the student's COA if they meet eligibility requirements. For graduate or professional students, there are also Direct PLUS Loans available. These loans do not have a fixed limit but cannot exceed the COA minus other aid.\n\nIt’s clear that these loan types are designed to help cover various financial gaps in education costs, and different eligibil

Finally, we can evaluate the new chain on the same test set!

#### 🎬 Study Note
Using this cell as a handy place to preserve the eval URL

Empified eval run:
View the evaluation results for experiment: 'impressionable-distribution-50' at:

https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/f76e0c37-88b6-4b5b-aa85-328dd0d8cc9e/compare?selectedSessions=8c48bb0a-c2d5-440e-a963-21e3c2629b22



In [54]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'impressionable-distribution-50' at:
https://smith.langchain.com/o/6a26aaa0-1b14-4cdb-933f-5cb4da328a84/datasets/f76e0c37-88b6-4b5b-aa85-328dd0d8cc9e/compare?selectedSessions=8c48bb0a-c2d5-440e-a963-21e3c2629b22




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,How do Appendix A and Appendix B relate to dis...,Thank you for your thoughtful question—it's cl...,None,Appendix B provides detailed guidance on disbu...,0,0,1,3.748226,b3e96bf2-c09a-45d6-a9b5-98204a055b3f,9f0f2364-cbdf-4310-a7f1-877d9447de9c
1,Considering the inclusion of clinical work in ...,Thank you for your thoughtful question—it's cl...,None,The guidance in Volume 7 emphasizes that for c...,1,1,1,5.943755,cce523b4-ef78-4224-8388-4e6b0635c44b,3a2396dd-2810-48b9-a288-1b033ce9542a
2,Considering the detailed requirements outlined...,Thank you for your thoughtful question. From t...,None,The definitions and regulations in Volume 8 sp...,1,0,1,4.216502,299b336b-4d83-48a0-877d-ec05da134f61,3a99368d-06cb-431d-81a0-4d553122267a
3,How do Volume 2 and Volume 8 relate to the inc...,Thank you for your thoughtful question. Based ...,None,Volume 2 provides guidance on the inclusion of...,0,0,1,4.788720,f1963851-9e89-4b60-9eea-595a4931ad16,2fd2fcfe-7531-4463-a9e9-d0793b5abaf1
4,what program reqirements are in 34 CFR 668.3(b...,Thank you for your question. Based on the cont...,None,"According to the context, 34 CFR 668.3(b) spec...",0,0,1,2.624858,5b78942c-d3ff-4b9a-8e78-6968ece98d12,9aa9bbf9-b1b6-4551-bb8e-d12c48db68a5
5,what are the academic calendars and regulatory...,Thank you for your question! Based on the info...,None,the academic calendars must define academic ye...,1,1,1,6.219633,9838e976-6888-4d56-85f6-9cb1ddbc46cd,21ca2bf7-33e4-42fb-abf6-6315e3d8b678
6,How do the defined payment periods relate to t...,I understand you're looking to see how payment...,None,The defined payment periods are part of the ac...,1,1,1,5.470599,130599d8-2d03-4c99-9377-0c5341625d63,c490a612-0e38-4be8-9a96-b62752aeff6a
7,How do accelerated progression effects influen...,Thank you for your thoughtful question about h...,None,Accelerated progression in clock-hour or non-t...,1,1,1,6.464165,b4bb22e0-6cba-43e5-b323-b300e4da1308,0e83ee2d-8a54-473a-ae77-5972660d1659
8,Is the Federal Work-Study (FWS) program subjec...,Thank you for your thoughtful question about t...,None,"No, the payment period is applicable to all Ti...",1,1,1,7.344927,550b6659-d41d-4692-9e10-88615db57a89,7fd7adce-c903-49e0-9ecb-70b628690ff8
9,Could you explain the significance of Volume 8...,Thank you for your thoughtful question. I unde...,None,Inclusion of clinical work in a standard term ...,1,1,1,5.534007,96b6de78-cd88-4781-8acc-853c126432a8,ac638a62-a14e-4c58-909d-9ff08a49bccb


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

#### ✅ Response to Activity #3
The empathy_evaluator improved from 0% during the baseline (rag_chain) to 100% in the empathy_rag_chain evaluation.

As shown in [this screenshot.](./Screenshot_chain_comparison.png)

This was the expected result, since the initial chain had no instructions about empathy, whereas the “empified” chain was explicitly told to 
>“answer the question using empathy and kindness, and make sure the user feels heard.”

Nothing else changed in the chain definition, and nothing else changed in the metrics results.

Note: the empified chain has 24 examples vs. 12 in the baseline. I believe this is because the cell that populates the dataset ran twice. This does not change my analysis, as the empathy score was 100% in both runs.


